In [1]:
import numpy as np
import scipy.optimize as opt
import scipy.stats as sts
import sklearn.gaussian_process as gp

In [2]:
class BayesianOptimizer:
    def __init__(self,length=1.,variance=1.,noise=1e-6,n_restarts=9):
        self.n_restarts = n_restarts
        self.kernel = gp.kernels.ConstantKernel(constant_value=variance,constant_value_bounds=(1e-5,1e5))*\
                        gp.kernels.Matern(length_scale=length,length_scale_bounds=(1e-5,1e5),nu=1.5)+\
                        gp.kernels.WhiteKernel(noise_level=noise,noise_level_bounds=(1e-5,1e5))
        self.gpr = gp.GaussianProcessRegressor(alpha=0.,kernel=self.kernel,n_restarts_optimizer=self.n_restarts)
    def __expected_improvement(self,candidate,seen,epsilon):
        [prediction,std] = [c.ravel()[0] for c in self.gpr.predict(candidate,return_std=True)]
        best_so_far = np.min(self.gpr.predict(seen))
        if std>0:
            z = (-prediction+best_so_far-epsilon)/std
            return (-prediction+best_so_far-epsilon)*sts.norm.cdf(z)+std*sts.norm.pdf(z)
        return 0.
    def optimize(self,n_iterations,initial1,initial2,bounds,apply,cost_function,*cost_args):
        '''
            Example Inputs
            regression_coefficient,learning_rate,num_neurons,batch_size
            initial1 -> [0.0,0.005,6,64]
            initial2 -> [0.5,0.01,12,32]
            bounds -> [(0.,3.),(0.0005,0.02),(5,15),(32,256)]
            apply -> [None,None,round,round]
        '''
        cost = np.array([cost_function(*initial1,*cost_args),cost_function(*initial2,*cost_args)])
        x = np.array([initial1,initial2])
        
        
        def propose_candidate(x,bounds,epsilon):
            def negative_expected_improvement(candidate):
                return -self.__expected_improvement(np.array([candidate]),x,epsilon)
            min_val = 10e5
            best_x = None
            for _ in range(self.n_restarts):
                initial_candidate = [np.random.uniform(low=l,high=u) for (l,u) in bounds]
                res = opt.minimize(negative_expected_improvement,initial_candidate,bounds=bounds,method='L-BFGS-B')
                if res.success:
                    if res.fun < min_val:
                        min_val = res.fun
                        best_x = res.x
            return best_x
        
        for _ in range(n_iterations):
            self.gpr.fit(x,cost[:,None])
            
            next_x = [fnc(val) if fnc is not None else val for val,fnc in zip(propose_candidate(x,bounds,1e-8),apply)]
            next_cost = cost_function(*next_x,*cost_args)
            x = np.concatenate([x,np.array([next_x])],0)
            cost = np.concatenate([cost,np.array([next_cost])])
        
        return [fnc(val) if fnc is not None else val for val,fnc in zip(x[np.argmin(cost)],apply)]